In [29]:
import netwolf_lib as nfb
import os
import socket
from math import ceil
import threading
from concurrent.futures import ThreadPoolExecutor
print('import library')



import library


In [3]:
for i in range(-1):
    print(i)

In [7]:
class AddressIp:

    def __init__(self, ip, pn, proxy_ip, proxy_pn):
        self.ip = ip
        self.pn = pn
        self.proxy_ip = proxy_ip
        self.proxy_pn = proxy_pn
    def __str__(self):
        return (f'ip:{self.ip},' + 
              f'pn:{self.pn},' +
              f'proxy_ip:{self.proxy_ip},' +
              f'proxy_pn:{self.proxy_pn}')
    

In [12]:
a = AddressIp('454', 4,5, 4)
c =AddressIp('454', 4,5, 4)
b ={a:'ehl'}
print(b[c])
print(a)

KeyError: <__main__.AddressIp object at 0x00000272D6BA2278>

In [18]:
import socket
address = socket.gethostbyname(socket.gethostname())
udp_server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
udp_server.bind((address, 6000))
from time import sleep
sleep(6)
print('hi')


hi
